# Build AML Pipeline with azureml modules

In this tutorial you will learn how to work with Azure ML module:

1. Setup enrivonment - install module CLI and module/pipeline SDK
2. Register a few sample modules into your aml workspace using CLI
3. Use module/pipeline SDK to create a pipeline with modules registered in step 2

## Prerequisite
* Install Azure CLI, please follow [the Azure CLI installation instructions](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) to install.
* Install docker desktop from [here](https://www.docker.com/products/docker-desktop) 

## Setup environment
* Install Azure CLI AML extension which includes the _module_ command group
* Install Azure ML SDK including the APIs to work with _module_ and _pipeline_

In [11]:
CLI_SDK_VERSION=19657780

In [ ]:
!az extension remove -n azure-cli-ml 

# Install local version of azure-cli-ml (which includes `az ml module` commands)
!az extension add --source https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION/azure_cli_ml-0.1.0.$CLI_SDK_VERSION-py3-none-any.whl --pip-extra-index-urls https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --yes 

In [ ]:
# Verify the availability of `az ml module` commands
#!az ml pipeline -h
!az ml module -h

In [ ]:
# Install azureml-sdk with Pipeline, Module
# Important! After install succeed, need to restart kernel

%config IPCompleter.greedy=True 
!pip install azureml-pipeline-wrapper[notebooks]==0.1.0.$CLI_SDK_VERSION --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --user --upgrade 

## Register azureml module

You can manage AML module through [azure-cli-ml](https://aka.ms/moduledoc) or [ml.azure.com](https://ml.azure.com/). <br>

Module could be registered from:
- local path
- public Github url
- Azure DevOps build artifacts

Azureml module support multiple module type:
- Basic python module
- Mpi module
- Parallel run module
- Hdi module (pending on backend support)

In [12]:
# you need to configure your ws information here

subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22' #'4aaa645c-5ae2-4ae9-a17a-84b9023bc56a'#'74eccef0-4b8d-4f83-b5f9-fa100d155b22'
workspace_name = 'lisal-amlservice' #'itp-pilot'#'kubeflow_ws_2' #'lisal-amlservice'
resource_group = 'lisal-dev' #'itp-pilot-ResGrp'#'kubeflow-demo' #'lisal-dev'

# Specify available aml compute in workspace
pipeline_compute = 'always-on-ds2v2' #'k80-16-a'#'kubeflow-aks' #'always-on-ds2v2'

In [ ]:
# Configure your aml workspace 

!az login 
!az account set -s $subscription_id 
!az ml folder attach -w $workspace_name -g $resource_group 

# Configure global .amlignore, it's designed for register module from local development environment
# !az configure --defaults module_amlignore_file=./.amlignore

In [ ]:
# Register azureml modules from github url

!az ml module register --spec-file=https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/mpi_train.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/score.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/eval.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/compare2.yaml --set-as-default-version

In [ ]:
# list available custom module in aml workspace
!az ml module list -o table 

## Create pipeline
You can build pipeline through SDK experience, or drag-n-drop way through [Designer](https://ml.azure.com/visualinterface?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1&flight=cm,nml,newGraphDetail,newGraphAuthoring,all&tid=72f988bf-86f1-41af-91ab-2d7cd011db47) in workspace portal

The new SDK:
* Symplified the syntax to provide consistent experience with drag-n-drop
* Support intellisense and docstring, free you to work with dict all the time
* Support creating a pipeline with unpublished module

In [13]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, Module, dsl

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

# get modules
# load module from ws registered modules
train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='MPI Train')
score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')
compare_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Compare 2 Models')

"""
# load module from local unregistered module
train_module_func = Module.from_yaml(ws, yaml_file='./train-score-eval/mpi_train.yaml')
score_module_func = Module.from_yaml(ws, yaml_file='./train-score-eval/score.yaml')
eval_module_func = Module.from_yaml(ws, yaml_file='./train-score-eval/eval.yaml')
compare_module_func = Module.from_yaml(ws, yaml_file='./train-score-eval/compare2.yaml')
"""

# get dataset
training_data_name = 'training_data'
test_data_name = 'test_data'

if training_data_name not in ws.datasets:
    print('Registering a training dataset for sample pipeline ...')
    train_data = Dataset.File.from_files(path=['https://dprepdata.blob.core.windows.net/demo/Titanic.csv'])
    train_data.register(workspace = ws, 
                              name = training_data_name, 
                              description = 'Training data (just for illustrative purpose)')
    print('Registerd')
else:
    train_data = ws.datasets[training_data_name]
    print('Training dataset found in workspace')

if test_data_name not in ws.datasets:
    print('Registering a test dataset for sample pipeline ...')
    test_data = Dataset.File.from_files(path=['https://dprepdata.blob.core.windows.net/demo/Titanic.csv'])
    test_data.register(workspace = ws, 
                          name = test_data_name, 
                          description = 'Test data (just for illustrative purpose)')
    print('Registered')
else:
    test_data = ws.datasets[test_data_name]    
    print('Test dataset found in workspace')


Training dataset found in workspace
Test dataset found in workspace


### dsl pipeline 
* 'Pipeline parameter' is exposed as pipeline function input parameter
* Pipeline output is the return of pipeline function

### module function
* module input can be set through set_inputs() or module initialization function
* module parameter can be set through set_parameter() or module initialization function
* module runsetting including compute, datastore, data mode and other runtime parameter are set through runsettings.configure()


In [14]:
# define a sub pipeline
@dsl.pipeline(name = 'A sub pipeline including train-score-eval', 
              description = 'train model and evaluate model perf')
def training_pipeline(input_data, test_data, learning_rate):
    train = train_module_func()

    train.set_inputs(training_data=input_data).set_parameters(learning_rate=learning_rate, max_epochs=5)
    train.runsettings.configure(process_count_per_node = 1, node_count = 1)

    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)
    
    return {'eval_output': eval.outputs.eval_output, 'model_output': train.outputs.model_output}

In [15]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'A dummy pipeline that trains multiple models and output the best one', 
              description = 'select best model trained with different learning rate',
              default_compute_target = pipeline_compute)
def dummy_automl_pipeline(input_data, test_data):
    train_and_evalute_model1 = training_pipeline(input_data, test_data, 0.01)
    train_and_evalute_model2 = training_pipeline(input_data, test_data, 0.02)
    
    compare = compare_module_func(
        model1=train_and_evalute_model1.outputs.model_output, 
        eval_result1=train_and_evalute_model1.outputs.eval_output,
        model2=train_and_evalute_model2.outputs.model_output,
        eval_result2=train_and_evalute_model2.outputs.eval_output
    )

    return {'best_model': compare.outputs.best_model, 'best_result': compare.outputs.best_result}

# create a pipeline
pipeline = dummy_automl_pipeline(input_data = train_data, test_data = test_data)

In [ ]:
# validate pipeline and visualize the graph
pipeline.validate()

In [ ]:
# save as a draft, then you can continue to modify the pipeline in AML Studio Designer page
pipeline.save(experiment_name = 'pipeline-with-azureml-module')

In [ ]:
# pipeline parameter can be override when submit pipeline
run = pipeline.submit(experiment_name='pipeline-with-azureml-module', tags={'mode':'module-SDK','SDK-version':f'{CLI_SDK_VERSION}'}, pipeline_parameters={'input_data':train_data,'test_data':test_data})
run.wait_for_completion()

In [21]:
pipeline.run(experiment_name='pipeline-local_run', show_output = True, show_graph = True)

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_4e064602-ea4b-4cfe-b154-9acc639597f9_widget', env_json='{"subscription…

Working dir: C:\Users\lisal\AppData\Local\Temp\pipeline-local_run\904a31dc-e103-4ab5-943e-b22108173ea3
RunId: 904a31dc-e103-4ab5-943e-b22108173ea3
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/pipeline-local_run/runs/904a31dc-e103-4ab5-943e-b22108173ea3?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/lisal-dev/workspaces/lisal-amlservice
Preparing pipeline dataset.
Prepared pipeline dataset.

C:\Users\lisal\AppData\Local\Temp\pipeline-local_run\904a31dc-e103-4ab5-943e-b22108173ea3\A sub pipeline including train-score-eval\MPI Train_8b293b1a-3821-439d-be83-85dcc76b8015\excutionlogs.txt
Preparing snapshot
Prepared snapshot, time elapsed 0:00:04.244330
Preparing experiment and run in run history
Prepared experiment and run in run history, time elapsed 0:00:04.019012
RunId: 8b293b1a-3821-439d-be83-85dcc76b8015
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/pipeline-local_run/runs/8b293b1a-3821-439d-be83-85dcc76b8015

'Completed'

### Load an unregistered module, and test if locally
* Support load module from local or github 
* Support use the module without register it to aml ws
* Use module.run() to test the module locally

In [17]:
# load unregistered module from github
copy_file_func = Module.from_yaml(ws, yaml_file='https://github.com/lisagreenview/hello-aml-modules/blob/master/parallel_copy_file/copy_files.yaml')
help(copy_file_func)

Help on function [module] Parallel Copy Files (namespace: microsoft.com/azureml/samples) in [module] Parallel Copy Files (namespace: microsoft:

[module] Parallel Copy Files (namespace: microsoft.com/azureml/samples)(*, input_folder:"['AnyDirectory']"=None)
    A sample Parallel module to copy files.
    
    Module yaml:
    ```yaml
    amlModuleIdentifier:
      namespace: microsoft.com/azureml/samples
      moduleName: Parallel Copy Files
      moduleVersion: 0.0.1
    isDeterministic: True
    jobType: Parallel
    description: A sample Parallel module to copy files.
    metadata:
      annotations:
        contact: Microsoft Coporation <xxx@microsoft.com>
        helpDocument: https://aka.ms/parallel-modules
        tags:
        - Sample
        - Parallel
    inputs:
    - name: Input Folder
      type: AnyDirectory
      optional: False
      argumentName: input_folder
    outputs:
    - name: Output Folder
      type: AnyDirectory
      description: Output images
      argumen

In [23]:
# create a module
# you need to prepare local test data to initialize the module
copy_file = copy_file_func(input_folder='./dummy_metrics')

copy_file.run(use_docker=True, track_run_history=True)

working dir is C:\Users\lisal\AppData\Local\Temp\9d19130d-cafb-46df-a41c-82961d11418e
Preparing snapshot
Prepared snapshot, time elapsed 0:00:03.079324
Preparing experiment and run in run history
Prepared experiment and run in run history, time elapsed 0:00:03.594204
RunId:fbee7cec-606b-4aa7-8f95-f6f33fadae3e
Link to Azure Machine Learning Portal:https://ml.azure.com/experiments/parallel_copy_files/runs/fbee7cec-606b-4aa7-8f95-f6f33fadae3e?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/lisal-dev/workspaces/lisal-amlservice
Pulling image from ACR (this may take a few minutes depending on image size)...

latest: Pulling from azureml/azureml_e31f6cb0e394a6368192ac2f6b563694
Digest: sha256:ab7b15e98b04b91890fdd961d40c3ee73419fdf3429dc291752e68321927112d
Status: Image is up to date for lisalamlservic4650097583.azurecr.io/azureml/azureml_e31f6cb0e394a6368192ac2f6b563694
Start running module Parallel Copy Files
Finish running module Parallel Copy Files, module run sta

'Completed'

### Create a new pipeline with unregistered module

new pipeline = dummy_automl_pipeline + copy_file_func

In [19]:
@dsl.pipeline(name='pipeline-with-azureml-module',default_compute_target = pipeline_compute)
def add_copy_file():
    compare_pipeline = dummy_automl_pipeline(input_data=train_data, test_data=test_data)
    copy_file_node = copy_file_func(input_folder=compare_pipeline.outputs.best_result)
    copy_file_node.runsettings.configure(node_count=1)

    return {**copy_file_node.outputs}

new_pipeline = add_copy_file()

### Run pipeline locally

* pipeline.run() support BasicPythonModule, ParallelRunModule and MpiModule 
* Local run and module node log will be uploaded and recorded in aml ws, and the running status will also be synced back to aml ws
* For mpi module, local run only support image with openmpi, intelmpi not support

In [22]:
os.chdir('D:\work\code\hello-aml-modules')
run = new_pipeline.run(experiment_name = 'pipeline-local_run', show_output = True, show_graph = True)
run

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_0ac1f0f3-e64a-4471-8976-a5a313fe2b96_widget', env_json='{"subscription…

Working dir: C:\Users\lisal\AppData\Local\Temp\pipeline-local_run\471c9933-2dfb-4710-9d7c-b774f48b4972
RunId: 471c9933-2dfb-4710-9d7c-b774f48b4972
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/pipeline-local_run/runs/471c9933-2dfb-4710-9d7c-b774f48b4972?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/lisal-dev/workspaces/lisal-amlservice
Preparing pipeline dataset.
Prepared pipeline dataset.
[Errno 2] No such file or directory: 'C:\\Users\\lisal\\AppData\\Local\\Temp\\pipeline-local_run\\471c9933-2dfb-4710-9d7c-b774f48b4972\\A dummy pipeline that trains multiple models and output the best one\\A sub pipeline including train-score-eval\\MPI Train_b8ae5db2-8cdd-45ff-ba38-cb6fa88952c8\\excutionlogs.txt'
MPI Train run failed, exception: [Errno 2] No such file or directory: 'C:\\Users\\lisal\\AppData\\Local\\Temp\\pipeline-local_run\\471c9933-2dfb-4710-9d7c-b774f48b4972\\A dummy pipeline that trains multiple models and output the best one\\A

'Failed'